In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [7]:
pd.set_option('display.max_columns', None)  # Показать все столбцы
pd.set_option('display.width', 1900)        # Установить ширину области отображения

In [8]:
# Загрузка данных
data = pd.read_csv('additional_variable_train.csv', sep=';') # Обратите внимание на путь файла

# Просмотр первых строк
# print(data.head())

In [10]:
# Выбираем признаки для анализа
X = data[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 
          'Ratio', 'Deviation from Average', 'Tool wear [min]', 'Sum_Parametr']]  # Признаки
y = data['Machine failure']  # Целевая переменная

In [11]:
# Масштабирование данных, так как ANOVA чувствителен к масштабу признаков
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
# Разделение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [17]:
# 1. Применяем метод ANOVA для выбора k лучших признаков
selector = SelectKBest(score_func=f_classif, k='all')  # 'all' выберет все признаки
X_new = selector.fit_transform(X_train, y_train)

# Получаем результаты анализа ANOVA
anova_scores = selector.scores_

# Для отбора по ANOVA
anova_score = cross_val_score(LogisticRegression(), X_new, y_train, scoring='accuracy').mean()

print(f"Точность модели для отбора по ANOVA: {anova_score:.4f}")

Точность модели для отбора по ANOVA: 0.9944


In [20]:
# 2. Применяем метод отбора признаков по дисперсии
var_threshold = VarianceThreshold(threshold=0.01)  # Устанавливаем порог дисперсии
X_var = var_threshold.fit_transform(X_train)

# Проверяем размерность данных после фильтрации
print(f"Размерность исходных данных: {X_train.shape}")
print(f"Размерность данных после отбора по дисперсии: {X_var.shape}")

# Для отбора по дисперсии
var_score = cross_val_score(LogisticRegression(), X_var, y_train, scoring='accuracy').mean()

print(f"\nТочность модели для отбора по дисперсии: {var_score:.4f}")

Размерность исходных данных: (95500, 8)
Размерность данных после отбора по дисперсии: (95500, 7)

Точность модели для отбора по дисперсии: 0.9844


In [ ]:
# Выводим результаты ВСе месте для сравнения
print(f"Точность модели для исходных данных: {original_score:.4f}")
print(f"\nТочность модели для отбора по дисперсии: {var_score:.4f}")
print(f"\nТочность модели для отбора по ANOVA: {anova_score:.4f}")

Точность модели для исходных данных: 0.9943

Точность модели для отбора по дисперсии: 0.9844

Точность модели для отбора по ANOVA: 0.9944


### Выводы из анализа:

1. **Точность модели для исходных данных (0.9943):**
   - Модель на исходных данных показывает очень высокую точность (99.43%). Это свидетельствует о том, что данные в изначальном виде достаточно информативны для предсказания целевой переменной.

2. **Точность модели для отбора по дисперсии (0.9844):**
   - После применения метода отбора признаков по дисперсии (Variance Threshold) точность модели немного снижается до 98.44%. Это указывает на то, что исключение признаков с низкой вариативностью не имеет существенного влияния на точность модели, но, возможно, уменьшает сложность модели, что может привести к лучшей обобщающей способности в других случаях.

3. **Точность модели для отбора по ANOVA (0.9944):**
   - Применение метода отбора признаков на основе ANOVA для оценки значимости признаков не изменяет точность модели, которая становится равной 99.44%. Это говорит о том, что признаки, выбранные с помощью ANOVA, не ухудшают модель и могут даже немного улучшить её точность.

### Общие выводы:
- **Метод ANOVA** оказался наиболее эффективным, так как он улучшил точность модели на 0.01% по сравнению с исходными данными.
- **Отбор по дисперсии** привел к небольшому снижению точности модели, что указывает на то, что некоторые признаки с низкой дисперсией могут все-таки быть полезны.
- В целом, методы отбора признаков по ANOVA и по дисперсии не привели к значительному ухудшению точности модели, но ANOVA может быть более предпочтительным для выбора информативных признаков.

### Рекомендация:
- Можно оставить модель с использованием **ANOVA**, так как она сохраняет или немного улучшает точность, при этом отбирая более значимые признаки.
- Для дальнейшего улучшения модели можно исследовать другие методы отбора признаков, такие как методы регуляризации или использование деревьев решений для оценки важности признаков.
